In [1]:
from eitprocessing.mixins.slicing import SelectByIndex
from dataclasses import dataclass, is_dataclass
from eitprocessing.eit_data.vendor import Vendor
from eitprocessing.eit_data.draeger import DraegerEITData
from eitprocessing.eit_data.eit_data_variant import EITDataVariant
from typing_extensions import Self

import os
import pytest
from pprint import pprint
import bisect
import numpy as np


In [2]:

draeger_file1 = '/home/dbodor/git/EIT-ALIVE/eitprocessing/tests/test_data/Draeger_Test3.bin'
data = DraegerEITData.from_path(draeger_file1)

# pprint(data)
data.time


array([75628.471, 75628.521, 75628.571, ..., 75719.321, 75719.371,
       75719.421])

In [3]:
cutoff = 10
print(data.label)
print(data[cutoff].label)
print(data[cutoff:cutoff+1].label)
print(data[0:cutoff].label)
print(data[cutoff:0].label)
print(data[cutoff:cutoff+2].label)


DraegerEITData_139828016111744
Frame (10) of <DraegerEITData_139828016111744>
Frame (10) of <DraegerEITData_139828016111744>
Slice (0-9) of <DraegerEITData_139828016111744>
No frames selected from <DraegerEITData_139828016111744>
Slice (10-11) of <DraegerEITData_139828016111744>


In [14]:
tp0 = 75628.471
tp2 = 75628.571

print(data.select_by_time(0,tp2).time)
print(data.select_by_time(0,tp2, start_inclusive=False).time)
print(data.select_by_time(0,tp2, end_inclusive=False).time)
print("")
print(data.select_by_time(tp0,tp2).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False, end_inclusive=False).time)

print("")
print('XXXXXXXXXXXXXXXXXXXXX')
print("")

tp0 = tp0-0.001
tp2 = tp2+0.001

print(data.select_by_time(0,tp2).time)
print(data.select_by_time(0,tp2, start_inclusive=False).time)
print(data.select_by_time(0,tp2, end_inclusive=False).time)
print("")
print(data.select_by_time(tp0,tp2).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False, end_inclusive=False).time)

print("")
print('XXXXXXXXXXXXXXXXXXXXX')
print("")

tp0 = tp0+0.002
tp2 = tp2-0.002

print(data.select_by_time(0,tp2).time)
print(data.select_by_time(0,tp2, start_inclusive=False).time)
print(data.select_by_time(0,tp2, end_inclusive=False).time)
print("")
print(data.select_by_time(tp0,tp2).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False).time)
print(data.select_by_time(tp0,tp2, start_inclusive=False, end_inclusive=False).time)


[75628.471 75628.521 75628.571]
[75628.471 75628.521 75628.571]
[75628.471 75628.521]

[75628.471 75628.521 75628.571]
[75628.521 75628.571]
[75628.521]

XXXXXXXXXXXXXXXXXXXXX

[75628.471 75628.521 75628.571]
[75628.471 75628.521 75628.571]
[75628.471 75628.521 75628.571]

[75628.471 75628.521 75628.571]
[75628.471 75628.521 75628.571]
[75628.471 75628.521 75628.571]

XXXXXXXXXXXXXXXXXXXXX

[75628.471 75628.521]
[75628.471 75628.521]
[75628.471 75628.521]

[75628.521]
[75628.521]
[75628.521]
